### Understanding Option Valuation as a Dynamical System using PySINDy

In [1]:
import numpy as np
import pysindy as ps
import pandas as pd
from datetime import datetime, timedelta,date
import matplotlib.pyplot as plt

from sklearn.linear_model import Lasso, Ridge, ElasticNet, Lars

### Import our data which is publicly available from Robinstocks

In terms of our dataset we used daily close values for a call on the S&P 500 Index expirying 09/30/2022 since listing

The underlying close was also accessed via Robinstocks

In [206]:
calls=pd.read_csv('data/calls.csv')
puts=pd.read_csv('data/puts.csv')
underlying=pd.read_csv('data/underlying_data.csv')

calls.index=calls[calls.columns[0]]
calls=calls.drop(columns=calls.columns[0])

puts.index=puts[puts.columns[0]]
puts=puts.drop(columns=puts.columns[0])

underlying.index=underlying[underlying.columns[0]]
underlying=underlying.drop(columns=underlying.columns[0])

In [207]:
calls['underlying']=underlying['price']
calls.index=[datetime.strptime(i,"%Y-%m-%d") for i in calls.index]
calls['time_to_exp']=[(datetime(2022,9,30)-i).days/365 for i in calls.index]
calls=calls[calls['time_to_exp']<0.8]
calls=calls.sample(frac=0.25).sort_index()[::-1]


In [337]:
calls

,symbol,strike,price,underlying,time_to_exp
2022-07-29,SPY,400.0,20.140,409.785,0.172603
2022-07-25,SPY,400.0,12.580,395.660,0.183562
2022-07-22,SPY,400.0,13.535,397.005,0.191781
2022-07-11,SPY,400.0,10.280,385.040,0.221918
2022-07-08,SPY,400.0,11.405,387.970,0.230137
2022-07-01,SPY,400.0,8.935,378.900,0.249315
2022-06-30,SPY,400.0,8.475,376.745,0.252055
2022-06-27,SPY,400.0,14.240,389.820,0.260274
2022-06-09,SPY,400.0,23.915,405.390,0.309589
2022-06-08,SPY,400.0,28.220,412.575,0.312329


### Black Scholes Partial Differential Equation


$$\frac{dv}{dt}+\frac{1}{2}\sigma^2 S^2\frac{d^2V}{dS^2}+rS\frac{dV}{dS}-rv$$


Where risk-free rate and volatility are a constant

# Calls

### PDEFIND from PySINDy

In [338]:
# Define PDE library that is linear in u and defined up to second derivatives spacially
library_functions = [lambda x: x, lambda x: x ]
library_function_names = [lambda x: x, lambda x: x+x ]
pde_lib = ps.PDELibrary(
    library_functions=library_functions,
    function_names=library_function_names,
    derivative_order=2, # constrain up to second-order derivatives 
    spatial_grid=calls['underlying'].values, # underlying values 
    include_bias=False, # assume there is not a drift process 
    is_uniform=False, # not uniformly sampled in price
    periodic=False# uniformly sampled in time
).fit(calls['price'].values.reshape(-1,1))
print("2nd order derivative library: ")
print(pde_lib.get_feature_names()) # differential operators we will be fitting a regression equation on

2nd order derivative library: 
['x0', 'x0x0', 'x0_1', 'x0_11', 'x0x0_1', 'x0x0x0_1', 'x0x0_11', 'x0x0x0_11']


### Least Angle Regression for variable selection


In [339]:
lars_optimizer=Lars(n_nonzero_coefs=5,eps=1e-6,fit_intercept=False)

smooth_fd=ps.SmoothedFiniteDifference(smoother_kws={'window_length': 5})

In [340]:
model = ps.SINDy(feature_library=pde_lib, optimizer=lars_optimizer)

model.fit(calls['price'].values.reshape(-1,1), t=calls['time_to_exp'].values)
model.print()

(x0)' = 5.946 x0 + -0.431 x0x0_1 + -0.239 x0x0x0_1


### LASSO Variable Selection

In [341]:
lasso_optimizer= Lasso(alpha=100.0,fit_intercept=False,max_iter=50000,tol=1e-4)

In [342]:
model = ps.SINDy(feature_library=pde_lib, optimizer=lasso_optimizer )

model.fit(calls['price'].values.reshape(-1,1), t=calls['time_to_exp'].values)
model.print()

(x0)' = 5.946 x0 + -0.215 x0x0_1 + -0.120 x0x0x0_1 + -0.215 x0x0_11 + -0.120 x0x0x0_11
